In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI
from dotenv import load_dotenv
import os
import random
from storysim import StorySimulator
import pandas as pd

/homes/ngetach/miniconda/envs/project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_KEY')


In [4]:
# This example is the new way to use the OpenAI lib for python
client = OpenAI()

def prompt_gpt(prompt, model):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
    )
    
    return response.choices[0].message.content

In [5]:
# graph = {
#         "hole_1": ["hole_2", "field"],
#         "hole_2": ["hole_1", "hole_3"],
#         "hole_3": ["hole_2", "hole_4"],
#         "hole_4": ["hole_3", "field"],
#         "field": ["hole_1", "hole_4"]
#     }

# events = {
        
#         7:{"name": "cross_paths","actors": ["Alice", "Bob"], "location": "hole_2"}, 
#         8: {"name": "exclusive_random", "actors":["Alice", "Bob"], "stop": 10 },
#         13: {"name": "mislead", "actors":["Alice", "Bob"]}
#     }


# sim = StorySimulator(
#         people=["Alice", "Bob", "Charlie", "Danny"],
#         locations=["hole_1", "hole_2", "hole_3", "hole_4", "field"],
#         relation="jumps_in",
#         trial_seed=10,
#         params={'prompt': '3', 'type': 'cot'},
#         graph=graph,
#         events=events
#     )

### Experiment Setup

In [6]:
def mislead_experiment(actors, locs, g, mislead, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[11] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[12] = {"name": "exclusive_random", "actors": poi, "stop": 12 + mislead}
    event_dict[12 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[12 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

def spaced_mislead_experiment(actors, locs, g, mislead, length):
    event_dict = {}
    poi = random.sample(actors, 2)
    loc = random.sample(locs, 1)
    label = random.choice([l for l in g[loc[-1]] if l != loc[-1]])
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc}
    event_dict[16] = {"name": "exclusive_random", "actors": poi, "stop": 17}
    event_dict[17] = {"name":"move", "actor":poi[-1],"location":label}
    event_dict[18] = {"name": "exclusive_random", "actors": poi, "stop": 18+mislead}
    event_dict[18+mislead] = {"name": "mislead", "actors": poi}
    event_dict[18 + mislead +1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def number_of_moves_experiment(actors, locs, g, length):
    poi = random.sample(actors, 2)
    loc = random.sample(locs,1)
    num_moves = 0
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc, "path_type":"same"}
    prev = loc[0]
    movement = []
    for i in range(1,num_moves+1):
        new_loc = random.choice([l for l in g[prev] if l not in loc])
        movement.append(new_loc)
        event_dict[10+i] = {"name":"move", "actor":poi[-1], "location": new_loc}
        prev = new_loc
    #event_dict[10+num_moves+1] = {"name": "mislead", "actors": poi}
    label =  movement[0]
    event_dict[10+num_moves+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, label, experiment_info

def second_order_tom_experiment(actors, locs, g, length):
    poi = random.sample(actors, 3)
    loc_1 = random.sample(locs,1)
    loc_2 = random.sample(g[loc_1[0]], 1)
    loc_3 = random.sample([l for l in g[loc_2[0]] if l != loc_1[0]])
    event_dict = {}
    event_dict[10] = {"name": "cross_paths","actors": poi, "location": loc_1, "path_type":"same"}
    event_dict[16] = {"name": "cross_paths","actors": poi[1:], "location": loc_1, "path_type":"same"}
    event_dict[17] = {"name":"move", "actor":poi[-1],"location":loc_3}
    event_dict[18] = {"name": "exclusive_random", "actors": poi, "stop": length}
    

def cross_path_overlap(actors, locs, g, mislead, length, n):
    poi = random.sample(actors, n)
    loc = random.sample(locs, 1)
    second_loc = random.choice(g[loc[0]])
    event_dict = {}
    event_dict[15] = {"name": "cross_paths","actors": poi, "location": loc, "path_type": "same"}
    event_dict[16] = {"name":"move", "actor":poi[-1], "location": second_loc}
    event_dict[17] = {"name": "exclusive_random", "actors": poi, "stop": 17 + mislead}
    event_dict[17 + mislead] = {"name": "mislead", "actors": poi}
    event_dict[17 + mislead+1] = {"name": "exclusive_random", "actors": poi, "stop": length}
    experiment_info = {'cross path location': loc[0], 'poi':poi}
    return event_dict, second_loc, experiment_info

In [7]:
def find_k_unique_paths(graph, start, end, k):
    def dfs(node, path, visited, paths):
        if len(paths) >= k:  # Stop early if we found k paths
            return
        
        if node == end:  # If reached destination, store the path
            paths.append(list(path))
            return

        for neighbor in graph.get(node, []):  # Explore neighbors
            if neighbor not in visited:
                visited.add(neighbor)
                path.append(neighbor)

                dfs(neighbor, path, visited, paths)

                # Backtrack
                visited.remove(neighbor)
                path.pop()

    paths = []
    dfs(start, [start], {start}, paths)  # Start DFS
    return [len(p)-1 for p in paths[:k]],[p[1:] for p in paths[:k]]  # Return up to k paths

### Actual Experiment setup

In [8]:
df = pd.DataFrame({'Story':[], 'Label':[], 'P1':[], 'P2':[]})

In [14]:


possible_people = ["Alice", "Bob", "Charlie", "Danny", "Edward", "Frank", "Georgia", "Hank", "Isaac", "Jake", "Kevin"]
num_people = 7
# graph = {
#         "hole_1": ["hole_2", "field"],
#         "hole_2": ["hole_1", "hole_3"],
#         "hole_3": ["hole_2", "hole_4"],
#         "hole_4": ["hole_3", "field"],
#         "field": ["hole_1", "hole_4"]
#     }


# graph = { 
#     "hole_1": ["hole_2", "field", "hole_5", "hole_6", "hole_4"],
#     "hole_2": ["hole_1", "hole_3", "field", "hole_5", "hole_7"],
#     "hole_3": ["hole_2", "hole_4", "field", "hole_8", "hole_1"],
#     "hole_4": ["hole_3", "hole_5", "hole_1", "hole_9", "field"],
#     "hole_5": ["hole_4", "hole_1", "hole_2", "hole_10", "field"],
#     "hole_6": ["hole_1", "hole_7", "hole_9", "field"],
#     "hole_7": ["hole_6", "hole_2", "hole_8", "field"],
#     "hole_8": ["hole_7", "hole_3", "hole_9", "field"],
#     "hole_9": ["hole_8", "hole_4", "hole_6", "hole_10", "field"],
#     "hole_10": ["hole_9", "hole_5", "field", "hole_7"],
#     "field": ["hole_1", "hole_2", "hole_3", "hole_4", "hole_9", "hole_10"]
# }

graph = { 
    "room_1": ["room_2", "the_hallway","room_5"],
    "room_2": ["room_1", "room_3","the_hallway"],
    "room_3": ["room_2", "room_4","the_hallway"],
    "room_4": ["room_3", "room_5","room_1"],
    "room_5": ["room_4", "room_1","room_2"],
    "the_hallway": ["room_1", "room_4","room_2"]
}

graph_2 = {
    "triangle_on_the_wall": ["square_on_the_wall", "rectangle_on_the_wall", "circle_on_the_wall"],
    "square_on_the_wall": ["triangle_on_the_wall", "rectangle_on_the_wall", "circle_on_the_wall"],
    "rectangle_on_the_wall": ["triangle_on_the_wall", "square_on_the_wall", "circle_on_the_wall"],
    "circle_on_the_wall": ["triangle_on_the_wall", "square_on_the_wall", "rectangle_on_the_wall"]
}


locations = list(graph.keys())
story_length = 25
num_trials = 40
mislead_distance = 3

random.seed(25)

event_dict, label, experiment_dict = cross_path_overlap(possible_people[:num_people], locations[:-1], graph, mislead_distance, story_length, 5)


print(experiment_dict['poi'])
sim = StorySimulator(
    people=possible_people[:num_people],
    locations=locations,
    relation="enters",
    params={'prompt': '3', 'type': 'cot'},
    graph=graph,
    events=event_dict
)

sim2 = StorySimulator(
    people=possible_people[:num_people],
    locations=list(graph_2.keys()),
    relation="draws_a",
    params={'prompt': '3', 'type': 'cot'},
    graph=graph_2,
    events=None
)

r2 = sim2.run_simulation(10)
print(r2)

res = sim.run_simulation(story_length)

print(res)


story = sim.formal_to_story(res)
# d = {'Story':[], 'Label':[], 'P1':[], 'P2':[]}
# d['P1'] = ",".join(experiment_dict['poi'][:-1]) if len(experiment_dict['poi'][:-1]) > 1 else experiment_dict['poi'][:-1]
# #print(d["P1"])
# d['P2'] = experiment_dict['poi'][-1]
# d['Story'].append(story)
# # d['Label'].append(movement[0])
# d['Label'].append(label)
# df = pd.concat([df, pd.DataFrame(d)])

['Danny', 'Alice', 'Bob', 'Charlie', 'Georgia']
['draws_a(Georgia, square_on_the_wall, 0)\n', 'draws_a(Alice, square_on_the_wall, 1)\n', 'draws_a(Edward, square_on_the_wall, 2)\n', 'draws_a(Alice, circle_on_the_wall, 3)\n', 'draws_a(Georgia, triangle_on_the_wall, 4)\n', 'draws_a(Edward, circle_on_the_wall, 5)\n', 'draws_a(Frank, triangle_on_the_wall, 6)\n', 'draws_a(Edward, rectangle_on_the_wall, 7)\n', 'draws_a(Charlie, triangle_on_the_wall, 8)\n', 'draws_a(Edward, square_on_the_wall, 9)\n']
['enters(Frank, room_2, 0)\n', 'enters(Edward, room_2, 1)\n', 'enters(Danny, room_4, 2)\n', 'enters(Charlie, room_4, 3)\n', 'enters(Edward, the_hallway, 4)\n', 'enters(Edward, room_2, 5)\n', 'enters(Bob, room_4, 6)\n', 'enters(Edward, room_1, 7)\n', 'enters(Edward, the_hallway, 8)\n', 'enters(Frank, room_3, 9)\n', 'enters(Frank, the_hallway, 10)\n', 'enters(Alice, room_4, 11)\n', 'enters(Frank, room_4, 12)\n', 'enters(Edward, room_4, 13)\n', 'enters(Frank, room_3, 14)\n', 'enters(Georgia, room_4, 

In [15]:
r2

['draws_a(Georgia, square_on_the_wall, 0)\n',
 'draws_a(Alice, square_on_the_wall, 1)\n',
 'draws_a(Edward, square_on_the_wall, 2)\n',
 'draws_a(Alice, circle_on_the_wall, 3)\n',
 'draws_a(Georgia, triangle_on_the_wall, 4)\n',
 'draws_a(Edward, circle_on_the_wall, 5)\n',
 'draws_a(Frank, triangle_on_the_wall, 6)\n',
 'draws_a(Edward, rectangle_on_the_wall, 7)\n',
 'draws_a(Charlie, triangle_on_the_wall, 8)\n',
 'draws_a(Edward, square_on_the_wall, 9)\n']

In [16]:
sim2.formal_to_story(r2)

'Georgia draws a square_on_the_wall. Alice draws a square_on_the_wall. Edward draws a square_on_the_wall. Alice draws a circle_on_the_wall. Georgia draws a triangle_on_the_wall. Edward draws a circle_on_the_wall. Frank draws a triangle_on_the_wall. Edward draws a rectangle_on_the_wall. Charlie draws a triangle_on_the_wall. Edward draws a square_on_the_wall'

In [140]:
df.head()

,Story,Label,P1,P2
0,Edward enters room_2. Charlie enters room_4. B...,room_3,"Danny,Alice,Bob,Charlie",Georgia
0,Edward enters room_1. Charlie enters room_4. E...,room_5,"Alice,Danny,Charlie,Bob",Georgia
0,Danny enters room_1. Alice enters room_4. Bob ...,room_2,"Georgia,Alice,Charlie,Bob",Edward
0,Frank enters room_1. Frank enters the_hallway....,room_5,"Alice,Edward,Charlie,Georgia",Bob
0,Edward enters room_4. Bob enters room_4. Charl...,the_hallway,"Georgia,Edward,Charlie,Bob",Danny


In [141]:
for i ,row in df.iterrows():
    print("\n".join(row['Story'].split(".")))
    print('---')

Edward enters room_2
 Charlie enters room_4
 Bob enters room_4
 Frank enters room_2
 Frank enters the_hallway
 Edward enters the_hallway
 Edward enters room_2
 Alice enters room_4
 Frank enters room_4
 Frank enters room_3
 Edward enters the_hallway
 Danny enters room_4
 Frank enters room_2
 Edward enters room_4
 Edward enters room_1
 Georgia enters room_4
 Georgia enters room_3
 Edward enters room_5
 Edward enters room_2
 Edward enters room_1
 Georgia enters room_2
 Frank enters the_hallway
 Frank enters room_4
 Frank enters room_1
 Frank enters the_hallway
---
Edward enters room_1
 Charlie enters room_4
 Edward enters the_hallway
 Edward enters room_1
 Frank enters room_4
 Frank enters room_5
 Frank enters room_1
 Danny enters room_4
 Frank enters the_hallway
 Frank enters room_4
 Bob enters room_4
 Frank enters room_5
 Alice enters room_4
 Edward enters the_hallway
 Frank enters room_1
 Georgia enters room_4
 Georgia enters room_5
 Edward enters room_2
 Edward enters the_hallway
 Edw